# Semantic Search using the Inference API with the Hugging Face Inference Endpoints Service

TODO [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/TODO)


Learn how to use the [Inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/inference-apis.html) with the Hugging Face Inference Endpoint service for semantic search.

# 🧰 Requirements

For this example, you will need:

- An Elastic deployment:
   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook))

- Elasticsearch 8.14 or above.
   
- A paid [Hugging Face Inference Endpoint](https://huggingface.co/docs/inference-endpoints/guides/create_endpoint) is required to use the Inference API with 
the Hugging Face Inference Endpoint service.

# Create Elastic Cloud deployment or serverless project

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

# Install packages and connect with Elasticsearch Client

To get started, we'll need to connect to our Elastic deployment using the Python client (version 8.12.0 or above).
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`

In [1]:
!pip install elasticsearch
%pip install datasets


[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Next, we need to import the modules we need. 🔐 NOTE: getpass enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [2]:
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time
from timeit import default_timer as timer
import datasets
import csv

/Users/mh/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [3]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=120, 
    max_retries=10,
    retry_on_timeout=True
)

### Test the Client
Before you continue, confirm that the client has connected with this test.

In [4]:
print(client.info())

{'name': 'instance-0000000001', 'cluster_name': 'dd945ac31a6c456686ee5117f1ec38a2', 'cluster_uuid': 'puJnzmQtThKzbfH1e3_bVw', 'version': {'number': '8.14.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd55f984299e0e88dee72ebd8255f7ff130859ad0', 'build_date': '2024-07-07T22:04:49.882652950Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Refer to [the documentation](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect to a self-managed deployment.

Read [this page](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect using API keys.

<a name="create-the-inference-endpoint"></a>
## Create the inference endpoint object

Let's create the inference endpoint by using the [Create inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html).

You'll need an Hugging Face API key (access token) for this that you can find in your Hugging Face account under the [Access Tokens](https://huggingface.co/settings/tokens).

You will also need to have created a [Hugging Face Inference Endpoint service instance](https://huggingface.co/docs/inference-endpoints/guides/create_endpoint) and noted the `url` of your instance. For this notebook, we deployed the `multilingual-e5-small` model.

In [ ]:
API_KEY = getpass("Huggingface API key:  ")
try:
    client.inference.delete_model(inference_id="my_hf_endpoint_object")
except Exception as e:
    print(e)
client.inference.put_model(
    inference_id='my_hf_endpoint_object',
    body={
        "service": "hugging_face",
        "service_settings": {"api_key": API_KEY, 
                             "url": "<URL-TO-HUGGING-FACE-ENDPOINT>",
                            "similarity": "dot_product"
                            },
    },
    task_type="text_embedding"
)

In [ ]:
client.inference.inference(
     inference_id='my_hf_endpoint_object',
     input="this is the raw text of my document!"
     )

**IMPORTANT:** If you use Elasticsearch 8.12, you must change `inference_id` in the snippet above to `model_id`! 

#

## Create an ingest pipeline with an inference processor

Create an ingest pipeline with an inference processor by using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method. Reference the `inference_id` created above as `model_id` to infer on the data that is being ingested by the pipeline.

In [ ]:
client.ingest.put_pipeline(
    id="hf_pipeline",
    processors=[
        {
            "inference": {
                "model_id": "my_hf_endpoint_object",
                "input_output": {
                    "input_field": "text",
                    "output_field": "text_embedding",
                },
            }
        }
    ],
)

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the inference endpoint to be used. In this example, the inference ID is set to `my_hf_endpoint_object`. Use the inference ID you defined when created the inference task.
- `input_output`: Specifies input and output fields.
- `input_field`: Field name from which the `dense_vector` representation is created.
- `output_field`:  Field name which contains inference results. 

## Create index

The mapping of the destination index - the index that contains the embeddings that the model will create based on your input text - must be created. The destination index must have a field with the [dense_vector](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html) field type to index the output of the model we deployed in Hugging Face (`multilingual-e5-small`).

Let's create an index named `hf-endpoint-index` with the mappings we need.

In [ ]:
client.indices.create(
    index="hf-endpoint-index",
    settings = {
        "index": {
            "number_of_replicas": "1",
            "number_of_shards": "1",
            "default_pipeline": "hf_pipeline",
        }
    },
    mappings = {
    "properties": {
        "text": {"type": "text"},
        "text_embedding": {
            "type": "dense_vector",
            "dims": 384,
            "similarity": "dot_product",
        },
    },
    "_source": {"excludes": ["passage_embedding.predicted_value"]},
    }
)

## If you are using Elasticsearch serverless or v8.15+ then you will have access to the new `semantic_text` field
`semantic_text` has significantly faster ingest times and is recommended.

https://github.com/elastic/elasticsearch/blob/main/docs/reference/mapping/types/semantic-text.asciidoc

In [ ]:
client.indices.create(
    index="hf-semantic-text-index",
    mappings={
        "properties": {
            "infer_field": {
                "type": "semantic_text",
                "inference_id": "my_hf_endpoint_object"
            }
        }
    },
)

## Insert Documents

In this example, we want to show the power of using GPUs in Hugging Face's Inference Endpoint service by indexing millions of multilingual documents from the miracl corpus. The speed at which these documents ingest will depend on whether you use a semantic text field (faster) or an ingest pipeline (slower) and will also depend on how much hardware your rent for your Hugging Face inference endpoint. 

In [ ]:
langs = ['ar', 'bn', 'en', 'es', 'fa', 'fi', 'fr', 'hi', 'id', 'ja', 'ko', 'ru', 'sw', 'te', 'th', 'zh']


all_langs_datasets = [iter(datasets.load_dataset('miracl/miracl-corpus', lang)['train']) for lang in langs]

In [ ]:
MAX_BULK_SIZE = 1000
MAX_BULK_UPLOADS = 1000

sentinel = object()
for j in range():
    start = timer()
    try:
        time.sleep(1)
        documents = []
        while len(documents) < MAX_BULK_SIZE - len(all_langs_datasets):
            for ds in all_langs_datasets:
                text = next(ds, sentinel)
                if text is not sentinel:
                    documents.append(
                        {
                            "_index": "hf-endpoint-index",
                            "_source": {"text": text['text']},
                        }
                    )
                    # if you are using semantic text, use this append instead:
                    # documents.append(
                    #     {
                    #         "_index": "hf-semantic-text-index",
                    #         "_source": {"infer_field": text['text']},
                    #     }
                    # )

        try:
            response = helpers.bulk(client, documents, raise_on_error=False, timeout="60s")

            print("Doc set:", j, end="; ")
            end=timer()

        except Exception as e:
            print("exception:", str(e))
            time.sleep(30)
    except Exception as e:
        print(j, e)

## Semantic search

After the dataset has been enriched with the embeddings, you can query the data using [semantic search](https://www.elastic.co/guide/en/elasticsearch/reference/current/knn-search.html#knn-semantic-search). Pass a `query_vector_builder` to the k-nearest neighbor (kNN) vector search API, and provide the query text and the model you have used to create the embeddings.

In [ ]:
response = client.search(
    index="hf-endpoint-index",
    size=3,
    knn={
        "field": "text_embedding",
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "my_hf_endpoint_object",
                "model_text": "Fighting movie",
            }
        },
        "k": 10,
        "num_candidates": 100,
    },
)

for hit in response["hits"]["hits"]:
    doc_id = hit["_id"]
    score = hit["_score"]
    text = hit["_source"]["text"]
    print(f"Score: {score}\nText: {text}\n")
    

**NOTE:** The value of `model_id` in the `query_vector_builder` must match the value of `inference_id` you created in the [first step](#create-the-inference-endpoint).